# Modeling a Gas Network using JuMP and Plasmo
### Jordan Jalving & Victor Zavala

## Graph based Modeling (component based modeling)
Plasmo applies a graph-based abstraction to model interconnected systems using nodes and edges.  In this sense, models (e.g. JuMP models) can be associated with nodes and edges and then coupled by providing function handles (coupling functions) to nodes or edges within their respective graph.  The key to this abstraction is the use of subgraphs.  It is possible to set coupling functions for a node within the context of multiple graphs.  This allows general coupling formulations that can connect a node to its neighbors within the prescribed subgraph.

## Network Equations
Although our system of equations is described in terms of sets, we will implement them in terms of individual model components.

$\newcommand{\norm}[1]{\left\lvert#1\right\rvert}$
__Objective__ (Minimize system compressor cost; maximize gas delivery to all demands)

$
\min \varphi:=\int_{0}^T \left(\sum_{\ell \in \mathcal{L}_a}\alpha_{P}P(\tau) - \sum_{d \in \mathcal{D}}\alpha_{gas} f_{deliver}(\tau)\right)d\tau
$


__Isothermal Euler Equations__

$
\begin{align*}
      &\frac{\partial p_{\ell}(t,x)}{\partial t} + \frac{c^2}{A_{\ell}}\frac{\partial f_{\ell}(t,x)}{\partial x} = 0, \ \ \ell \in \mathcal{L}\\
      &\frac{\partial f_{\ell}(t,x)}{\partial t} + \frac{2c^2f_{\ell}(t,x)}{A_{\ell}p_{\ell}(t,x)}\frac{\partial f_{\ell}(t,x)}{\partial x} - \frac{c^2f_{\ell}(t,x)^2}{A_{\ell}p_{\ell}(t,x)^2}\frac{\partial p_{\ell}(t,x)}{\partial x}
          + A_{\ell}\frac{\partial p_{\ell}(t,x)}{\partial x} = -\frac{8c^2\lambda A_{\ell}}{\pi^2D_{\ell}^5}\frac{f_{\ell}(t,x)}{p_{\ell}(t,x)} \norm{f_{\ell}(t,x)}, \ \ \ell \in \mathcal{L}
    \end{align*}
$

__Compressor Equation__

$
\begin{align*}
&P_{\ell}(t)= c_p\cdot T\cdot f_{in,\ell}\left(\left(\frac{p_{in,\ell}(t)+\Delta p_{\ell}(t)}{p_{in,\ell}(t)}\right)^{\frac{\gamma-1}
{\gamma}}-1\right),\ell \in \mathcal{L}_a\\
\end{align*}
$

__Pipe Boundary Conditions__

$
\begin{align*}
&p_{\ell}(0,t)= p_{in,\ell}(t)+\Delta p_{\ell}(t),\ell \in \mathcal{L}_a\\
&p_{\ell}(0,t)= p_{in,\ell}(t),\ell \in \mathcal{L}_p\\
&p_{\ell}(L_{\ell},t)= p_{out,\ell}(t),\ell \in \mathcal{L}\\
\end{align*}
$

__Node Conservation__

$
\begin{align*}
&\sum_{\ell\in\mathcal{L}_n^{rec}}f_{out,\ell}(t)-\sum_{\ell \in\mathcal{L}_n^{snd}}f_{in,\ell}(t) +
\sum_{i\in\mathcal{S}_n}g_i(t) - \sum_{j\in \mathcal{D}_n}d_j^{gas}(t) = 0,\; n \in\mathcal{N}
\end{align*}
$

__Supply and Demand__

$
\begin{align*}
&f_{deliver,n}(t) \le f_{demand,n}(t),n \in \mathcal{D} \\
& \underline{s} \le g_n(t) \le \overline{s},n \in \mathcal{S}
\end{align*}
$

## Import JuMP and Ipopt

In [1]:
using JuMP
using Ipopt
using PyPlot

## The usual physical property data

In [2]:
eps= 0.025		             # pipe rugosity - [mm]
z= 0.80        			     # gas compressibility  - []
rhon=0.72         		     # density of air at normal conditions - [kg/m3]
R=8314.0       			     # universal gas constant [J/kgmol-K]
M=18.0    			         # gas molar mass [kg/kgmol]
pi=3.14         		     # pi
Tgas = 293.15      		     # reference temperature [K]
Cp = 2.34        		     # heat capacity @ constant pressure [kJ/kg-K]
Cv = 1.85        		     # heat capacity @ constant volume [kJ/kg-K]

#scaling factors
ffac=(1e+6*rhon)/(24*3600)                     # from scmx10-6/day to kg/s
ffac2=(3600)/(1e+4*rhon)                       # from kg/s to scmx10-4/hr
pfac=1e+5                                      # from bar to Pa
pfac2=1e-5                                     # from Pa to bar
dfac=1e-3                                      # from mm to m
lfac=1e+3;                                      # from km to m

#We will also use a global horizon variable!
horizon = 24*3600;

# Gas Network Component Models
Here we will define the component models for our gas network.  While there is certainly some effort required to come up with valid model components, we will see that the system as a whole becomes easier to analyze and extend.

### Gas Junction Component

In [3]:
################################
# Gas Node Component
################################
type JunctionData
    time_grid
    pressure_lower
    pressure_upper
end

function gasjunction(data)
    m = Model()
    time_grid = data.time_grid
    @variable(m,data.pressure_lower <= pressure[time_grid] <= data.pressure_upper, start = 60)
    @variable(m,supply[time_grid] >= 0)  #supply flow to gasnode
    @variable(m,deliver[time_grid] >= 0) #delivered flow from gas node
    return m
end;

### Gas Demand Component

In [4]:
####################################
# Demand Component
####################################
type DemandData
    time_grid
    cost
end

function gasdemand(data)
    cost = data.cost
    time_grid = data.time_grid
    m = Model()
    @variable(m,fdeliver[time_grid] >= 0, start = 100)
    @variable(m, demandcost)
    @variable(m, fdemand[time_grid] >= 0)
    @constraint(m, flowLimit[t = time_grid], fdeliver[t] <= fdemand[t])
    @constraint(m, integratedGasCost, demandcost == sum(cost*fdeliver[t] for t = time_grid))
    @objective(m, Min, demandcost)
    return m
end;

### Gas Supply Component

In [5]:
####################################
# Supply Component
####################################
type SupplyData
    time_grid
    cost
    fgen_lower
    fgen_upper
end

function gassupply(data)
    cost = data.cost
    time_grid = data.time_grid
    m = Model()
    @variable(m, data.fgen_lower <= fgen[time_grid] <= data.fgen_upper, start = 10)
    @variable(m,gencost[time_grid])
    @constraint(m,costconstraint[t = time_grid], gencost[t] == cost*fgen[t])
    return m
end;

## Creating a gas pipeline component

In [6]:
#Pipe data struct
type PipeData
    len
    diameter
    time_grid
    x_grid
    min_pressure
    max_pressure
    min_flow
    max_flow
end

#Compressor data struct
type CompData
    dp_min
    dp_max
    min_power
    max_power
    cost
end

### *Notice that we do not use sets of links.  Each pipeline is its own component encapsulating its own variables and equations*

Since pipeline components can be extremely diverse (various reduced models, active vs passive,etc...), it is helpful to define functions that manipulate a model.  We can then build up different pipeline models for different applications and set them to edges within our network.

### Base Pipe Equations

In [7]:
function basicpipeequations(m,pipedata)
    time_grid = pipedata.time_grid 
    x_grid = pipedata.x_grid
    min_pressure = pipedata.min_pressure
    max_pressure = pipedata.max_pressure
    min_flow = pipedata.min_flow
    max_flow = pipedata.max_flow
    @variable(m,pin[time_grid] >= 0, start = 60)
    @variable(m,pout[time_grid] >= 0, start = 60)
    @variable(m,fin[time_grid] >=0, start = 100)
    @variable(m,fout[time_grid] >= 0, start = 100)
    @variable(m, min_pressure <= px[time_grid,x_grid] <= max_pressure, start = 60)#start = 0.5*($data.min_pressure + $data.max_pressure)) #60   # link pressure profile - [bar]
    @variable(m, min_flow <= fx[time_grid,x_grid] <= max_flow, start = 10)#100 #start = 0.5*($data.min_flow + $data.max_flow))
    @constraint(m, flow_in[t = time_grid],  fx[t,1] == fin[t])
    @constraint(m, flow_out[t = time_grid], fx[t,x_grid[end]] == fout[t])
    return m
    end;

basicpipeequations (generic function with 1 method)

### Isothermal Euler Equations

In [8]:
function isothermaleulerequations(m,pipedata)
    #pipe parameters
    time_grid = pipedata.time_grid 
    x_grid = pipedata.x_grid
    diameter = pipedata.diameter
    len = pipedata.len
    x_grid = pipedata.x_grid
    time_grid = pipedata.time_grid
    area = (1/4)*pi*diameter*diameter
    lam = (2*log10(3.7*diameter/(eps*dfac)))^(-2)   #pipe friction coefficient
    c1 = (pfac2/ffac2)*(nu2/area)
    c2 = area*(ffac2/pfac2)
    c3 = area*(pfac2/ffac2)*(8*lam*nu2)/(pi*pi*diameter^5)
    dx = len / (length(x_grid) - 1)
    dt = horizon / length(time_grid)
    
    #variables and equations
    px = getvariable(m,px)
    fx = getvariable(m,fx)
    @variable(m, slack2[time_grid,x_grid] >= 0, start = 10)  #auxiliary variable
    @variable(m, slack3[time_grid,x_grid] >= 0, start = 10)  #auxiliary variable
    @variable(m, slack1[time_grid,x_grid] >= 0, start = 10)  #auxiliary variable for friction loss term
    @NLconstraint(m, slackeq1[t = time_grid, x = x_grid],  slack1[t,x]*px[t,x] - c3*fx[t,x]*fx[t,x] == 0)
    @NLconstraint(m, slackeq2[t = time_grid, x = x_grid],  slack2[t,x]*px[t,x] - 2*c1*fx[t,x] == 0)
    @NLconstraint(m, slackeq3[t = time_grid, x = x_grid],  slack3[t,x]*px[t,x]*px[t,x] - c1*fx[t,x]*fx[t,x] == 0)
    @constraint(m, press[t = time_grid[1:end-1], x = x_grid[1:end-1]], (px[t+1,x]-px[t,x])/dt + c1*(fx[t+1,x+1]-fx[t+1,x])/dx == 0 )
    @constraint(m, flow[t = time_grid[1:end-1], x = x_grid[1:end-1]], (fx[t+1,x]-fx[t,x])/dt == -slack2[t+1,x]*(fx[t+1,x+1]-fx[t+1,x])/dx +
                                    slack3[t+1,x]*(px[t+1,x+1]-px[t+1,x])/dx -c2*(px[t+1,x+1]-px[t+1,x])/dx - slack1[t+1,x])
    return m
    end;

isothermaleulerequations (generic function with 1 method)

### Compressor Equations

In [9]:
function compressorequations(m,compressdata)
    time_grid = compressdata.time_grid
    dp_min = compressdata.dp_min
    dp_max = compressdata.dp_max
    min_power = compressdata.min_power 
    max_power = compressdata.max_power
    cost = compressdata.cost
    fin = getvariable(m,:fin)
    pin = getvariable(m,:pin)
    @variable(m, dp_min <= dp[time_grid] <= dp_max, start = 10)
    @variable(m, min_power <= pow[time_grid] <= max_power, start = 500)
    @variable(m, powercost)
    @NLconstraint(m, powereqn[t = time_grid], pow[t] == c4*fin[t]*(((pin[t]+dp[t])/pin[t])^om-1))
    @constraint(m,boostcosteqn, powercost == sum(cost*pow[t] for t = time_grid)*dt/3600)
    @objective(m, Min, powercost)
    return m
end;

### Boundary Conditions

In [10]:
function activepressureboundary(m,pipedata)
    time_grid = pipe.time_grid
    px = getvariable(m,:px)
    pin = getvariable(m,:pin)
    dp = getvariable(m,:dp)
    pout = getvariable(m,:pout)
    @constraint(m,press_in[t = time_grid],  px[t,1] == pin[t] + dp[t])
    @constraint(m,press_out[t = time_grid], px[t,x_grid[end]] == pout[t])
    return m
end

activepressureboundary (generic function with 1 method)

In [11]:
function passivepressureboundary(m,pipedata)
    time_grid = pipe.time_grid
    px = getvariable(m,:px)
    pin = getvariable(m,:pin)
    pout = getvariable(m,:pout)
    @constraint(m,press_in[t = time_grid],  px[t,1] == pin[t])
    @constraint(m,press_out[t = time_grid], px[t,x_grid[end]] == pout[t])
    return m
end

passivepressureboundary (generic function with 1 method)

In [ ]:
function ssinitialconditions(m,pipedata)
    x_grid = pipedata.x_grid
    px = getvariable(m,:px)
    fx = getvariable(m,:fx)
    slack = getvariable(m,:slack)
    @constraint(m, mass_ss[t = 1, x = x_grid[1:end-1]], (fx[t,x+1] - fx[t,x]) == 0)
    @constraint(m, momentum_ss[t = 1, x = x_grid[1:end-1]], -c2*(px[t,x+1] - px[t,x])/dx - slack[t,x] == 0)
end

### Linepack Constraint

In [12]:
function linepack(m,pipedata)
    time_grid = pipedata.time_grid
    @variable(m,linepack[time_grid])
    @constraint(m,linepack_def[t = time_grid],linepack[t] == sum(fx[t,x] for x in x_grid)*dx)
    #Periodic terminal constraint
    @constraint(m,linepack_cons, linepack[time_grid[end]] >= linepack[time_grid[1]])

LoadError: syntax: incomplete: "function" at In[12]:1 requires end

## Pipeline Models

In [13]:
function euleractivelink(pdata,cdata)
    m = Model()
    m = basepipeequations(m,pdata)
    m = isothermaleulerequations(m,pdata)
    m = compressorequations(m,cdata)
    m = activepressureboundary(m,pipedata)
    m = ssinitialcondition(m,pipedata)
    m = linepackconstraint(m,pipedata)
    return m
end

euleractivelink (generic function with 1 method)

In [14]:
function eulerpassivelink(pdata)
    m = Model()
    m = basepipeequations(m,pdata)
    m = isothermaleulerequations(m,pdata)
    m = passivepressureboundary(m,pdata)
    m = ssinitialcondition(m,pdata)
    m = linepackconstraint(m,pdata)
    return m
end 

eulerpassivelink (generic function with 1 method)

## Coupling Functions

In [15]:
function couplesupplydemand(m::Model,graph,node)
    supply = getvariable(node,:supply)
    deliver = getvariable(node,:deliver)
    #couple supply and delivery to junction variables for supply and delivery
    @constraint(m,fsup[t = time_grid], supply[t]  ==  sum(getvariable(neighbors_in(graph,node)[i],:fgen)[t] for i = 1:n_neighbors_in(graph,node)))
    @constraint(m,fdel[t = time_grid], deliver[t] ==  sum(getvariable(neighbors_out(graph,node)[i],:fdeliver)[t] for i = 1:n_neighbors_out(graph,node)))
end

function couplegasjunction(m::JuMP.Model,graph,node)
    supply1 = getvariable(node,:supply)
    deliver1 = getvariable(node,:deliver)
    links_in = edges_in(graph,node)  #the links coming in at the higher level graph
    links_out = edges_out(graph,node)
    flow = @constraint(m,[t = time_grid], 0 == sum(getnodevariable(links_in[i],:fout)[t] for i = 1:length(links_in)) -
    sum(getnodevariable(links_out[i],:fin)[t] for i = 1:length(links_out)) + supply1[t] - deliver1[t])
end

function couplelink!(m::JuMP.Model,graph,edge)
    node_from = getconnectedfrom(edge)
    node_to = getconnectedto(edge)
    pin = getvariable(edge,:pin)
    pout = getvariable(edge,:pout)
    @constraint(m,pressure_in[t = time_grid],pin[t] == getvariable(node_from,:pressure)[t])
    @constraint(m,pressure_out[t = time_grid],pout[t] == getvariable(node_to,:pressure)[t])
end

couplelink! (generic function with 1 method)

# Create system models with Plasmo
## A single pipeline

![title](gas_coupling.png)

### Setup our data for the pipeline

In [16]:
#pipeline data
pipe_length = 30000
diameter = 0.92
time_grid = 1:48
x_grid = 1:10
min_pressure = 1
max_pressure = 100
min_flow = 1
max_flow = 300

#compressor data
dp_min = 1
dp_max = 20
min_power = 1
max_power = 100000 
comp_cost = 0.1
pdata = PipeData(pipe_length,diameter,time_grid,x_grid,min_pressure,max_pressure,min_flow,max_flow) #pipe data
cdata = CompData(dp_min,dp_max,min_power,max_power,comp_cost) #compressor data

CompData(1,20,1,100000,0.1)

### Set our data for the junctions, demand, and supply

In [17]:
#junction parameters
pressure_lower = 50
pressure_upper = 70

#demand parameters
gas_cost = -1000

#supply parameters
supply_cost = 0
fgen_lower = 0
fgen_upper = 600

jdata = JunctionData(time_grid,pressure_lower,pressure_upper)
ddata = DemandData(time_grid,gas_cost)
sdata = SupplyData(time_grid,supply_cost,fgen_lower,fgen_upper)

SupplyData(1:48,0,0,600)

## Create the Plasmo Graph

In [ ]:
using Plasmo
#Create a graph for the first junction
graph1 = PlasmoGraph()
j1 = add_node!(graph1)
s1 = add_node!(graph1)
add_edge!(graph1,s1,j1)

setmodel!(j1,gasjunction(jdata))
setmodel!(s1,gassupply(sdata))
setcouplingfunction!(graph1,j1,couplesupplydemand)

#Create a graph for the second junction
graph2 = PlasmoGraph()
j2 = add_node!(graph2)
d2 = add_node!(graph2)
add_edge!(graph2,j2,d2)
setmodel!(j2,gasjunction(jdata))
setmodel!(d2,gasdemand(ddata))
setcouplingfunction!(graph2,j2,couplesupplydemand)

#Create a graph containing both graph1 and graph2 as subgraphs
graph3 = PlasmoGraph()
add_subgraph!(graph3,graph1)
add_subgraph!(graph3,graph2)
pipeline = add_edge!(graph3,j1,j2)
setmodel!(pipeline,euleractivelink(pdata,cdata))
setcouplingfunction!(graph3,pipeline,couplelink)

#Set the junction couplings in graph3
setcouplingfunction!(graph3,j1,couplegasnode)
setcouplingfunction!(graph3,j2,couplegasnode)

## Solve the graph model

In [ ]:
graph_model = GraphModel(graph3)
#NOTE: We can continue to modify the graphmodel.  It contains the same index references to nodes and edges.  
#getvariable works the same way

graph_model.solver = IpoptSolver()
solve(graph_model)

## Look at the results

In [18]:
px = getvariable(pipeline,:px)
fx = getvariable(pipeline,:fx)

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in getvariable(::Function, ::Vararg{Any,N}) at ./deprecated.jl:30
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/jordan/.julia/v0.5/IJulia/src/execute_request.jl:156
 in eventloop(::ZMQ.Socket) at /home/jordan/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##13#19)() at ./task.jl:360
while loading In[18], in expression starting on line 1


LoadError: MethodError: no method matching getindex(::Base.#pipeline, ::Symbol)

# Modeling the 13 pipeline system with Plasmo

In [ ]:
network = PlasmoGraph()

#Add 13 subgraphs.  One for each junction in the network
for i = 1:14
    junction_graph = PlasmoGraph()
    node = add_node!(junction_graph)
    setmodel!(node,junction(jdata))
    setcouplingfunction!(junction_graph,node,couplesupplydemand)
    setcouplingfunction!(network,node,couplegasnode)
    add_subgraph!(network,junction_graph)
end

#The first junction has a supply.  
graph1 = getsubgraph(network,1) #We can get subgraphs by their index
junction1 = getnode(graph1,1)   #We can also get nodes by their index
supply_node = add_node!(graph1)
setmodel!(supply_node,gassuply(sdata))
add_edge!(graph1,supply_node,junction1)

#The last junction has a demand.
graph2 = getsubgraph(network,14)
junction2 = getnode(graph2,1)
demand_node = add_node!(graph2)
setmodel!(demand_node,gasdemand(ddata))
add_edge!(graph2,junction2,demand_node)

#Add pipelines between the junctions 
for j = 1:13
    edge = add_edge!(network,j,j+1)
    if j == 1 || 13
        setmodel!(edge,eulerpassivelink(pdata))
    else
        setmodel!(edge,euleractivelink(pdata,cdata))
    end
    setcouplingfunction!(network,edge,couplelink)
end

## Generate and solve the graph model

In [ ]:
graph_model = GraphModel(network)
graph_model.solver = IpoptSolver()
solve(graph)